In [0]:
!pip install imagehash

     |████████████████████████████████| 296kB 4.9MB/s 


In [0]:
from imagehash import *
from PIL import Image
import pandas as pd
import numpy as np

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!cp "/content/drive/My Drive/gsoc_images/images.zip" .

In [0]:
!cp "/content/drive/My Drive/gsoc_images/rkd_keys.csv" .

In [0]:
!unzip -qq images.zip

In [0]:
A = str(phash(Image.open('overzicht.jpg')))

In [0]:
B = str(phash(Image.open('rkdnotav.jpg')))

In [0]:
C = str(phash(Image.open('rkdimgnotav.jpg')))

In [0]:
D = str(phash(Image.open('rkdbadblur.png')))

In [0]:
def trim_df(df):
    df['IC1'] = df['Iconclass'].str[:3]
    df = df.groupby('Id').agg({'Iconclass':','.join,'IC1':set}).reset_index()
    df['IC1'] = df['IC1'].str.join(',')
    df["IMG"] = df["Id"].astype(str).__add__(".jpg")
    return df

In [0]:
#df = trim_df(pd.read_csv("rkd_keys.csv")[['Iconclass','Id']])
df = pd.read_csv("rkd_keys.csv")

In [0]:
b=[]
for i,id in enumerate(df['Id']):
    k = str(phash(Image.open(f'images/{id}.jpg')))
    if k in [A,B,C,D]:
        b.append(id)

In [0]:
len(np.unique(b))

In [0]:
tdf = df[~df["Id"].isin(b)]

In [0]:
kdf = trim_df(pd.read_csv("rkd_keys.csv")[['Iconclass','Id']])

In [0]:
ktdf = kdf[~kdf["Id"].isin(b)]

In [0]:
tdf[['Iconclass','Text','Download-Hash','Permalink','Id']].to_csv('rkd_keys_clean.csv')

In [0]:
len(ktdf)

10447

In [0]:
l = len(ktdf)
tov = [False] * l
tov[:int(.7 * l)] = [True] * int(.7 * l)

In [0]:
ktdf['Type'] = ['Train' if x else 'Valid' for x in tov]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [0]:
.7 * l

7312.9

In [0]:
ktdf['Type'].value_counts()

Train    7312
Valid    3135
Name: Type, dtype: int64

In [0]:
ktdf[["Id","Iconclass","Type"]].to_csv("rkd_cnn_train.csv")